Overpass Turbo Request: https://overpass-turbo.eu/s/1g8P

In [17]:
import requests
import io
import pandas as pd
import geojson


overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:csv(::"id",::type,"name",::lat,::lon,"wikidata","wikipedia","website","email","opening_hours";true; ";")];
area["ISO3166-1"="CH"][admin_level=2];
(node["historic"="castle"](area);
 way["historic"="castle"](area);
 rel["historic"="castle"](area);
);
out center;
"""
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.content

df = pd.read_csv(io.StringIO(data.decode('utf-8')), delimiter=";")

# https://stackoverflow.com/questions/47308984/how-to-use-python-with-pandas-exporting-data-to-geojson-for-google-map
# https://gist.github.com/mapcloud/036688121a87f960e35cc03077001e93
# https://shouland.com/pandas-to-geojson-multiples-points-features-with-python

# Replace nan to None
dataFrame = df.where((pd.notnull(df)), None)

def data2geojson(df):
    features = []
    insert_features = lambda X: features.append(
            geojson.Feature(geometry=geojson.Point((X["@lon"], X["@lat"])),
                            properties=dict(name=X["name"], wikidata=X["wikidata"], wikipedia=X["wikipedia"], website=X["website"], email=X["email"], opening_hours=X["opening_hours"])))
    df.apply(insert_features, axis=1)
    with open('castle.geojson', 'w', encoding='utf8') as fp:
        geojson.dump(geojson.FeatureCollection(features), fp, indent=4, sort_keys=True, ensure_ascii=False)

data2geojson(dataFrame)